In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/wineuci/Wine.csv


In [2]:
data=pd.read_csv('/kaggle/input/wineuci/Wine.csv')
data.columns= ['Type','Alcohol',
 'Malic acid',
 'Ash',
 'Alcalinity of ash',
 'Magnesium',
 'Total phenols',
 'Flavanoids',
 'Nonflavanoid phenols',
 'Proanthocyanins',
'Color intensity',
'1Hue',
'1OD280/OD315 of diluted wines',
'Proline']

In [3]:
data.head()

,Type,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,1Hue,1OD280/OD315 of diluted wines,Proline
0,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
1,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
2,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
3,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735
4,1,14.20,1.76,2.45,15.2,112,3.27,3.39,0.34,1.97,6.75,1.05,2.85,1450


In [4]:
X=data.iloc[:,1:]
y=data.iloc[:,0:1]
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.20,stratify=y,random_state=42,shuffle=True)

In [5]:
import h2o
from random import sample

train=pd.concat([X_train,y_train], axis=1)
test=pd.concat([X_test,y_test],axis=1)
print(train['Type'].value_counts())
print(test['Type'].value_counts())
print('Total train observations:'+str(len(train)))
print('Total test observations:'+str(len(test)))


test2=test.drop(columns='Type',axis=1)

import h2o
from h2o.automl import H2OAutoML
h2o.init()

X=list(train.columns[0:-1])
y=train.columns[-1]

train = h2o.H2OFrame(train)
train['Type']=train['Type'].asfactor()

aml_ti = H2OAutoML(max_runtime_secs= 120,max_models= 50, seed= 7,nfolds= 10)
aml_ti.train(x = X, y = y,
          training_frame = train)
          
#check the leaderboard
lb_ti = aml_ti.leaderboard
print(lb_ti)

test2 = h2o.H2OFrame(test2)
pred = aml_ti.leader.predict(test2)
prediction=pred.as_data_frame()

2    57
1    46
3    38
Name: Type, dtype: int64
2    14
1    12
3    10
Name: Type, dtype: int64
Total train observations:141
Total test observations:36
Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.9.1" 2020-11-04; OpenJDK Runtime Environment (build 11.0.9.1+1-Ubuntu-0ubuntu1.18.04); OpenJDK 64-Bit Server VM (build 11.0.9.1+1-Ubuntu-0ubuntu1.18.04, mixed mode, sharing)
  Starting server from /opt/conda/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpexbq6ekq
  JVM stdout: /tmp/tmpexbq6ekq/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpexbq6ekq/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.0.2
H2O_cluster_version_age:,27 days
H2O_cluster_name:,H2O_from_python_unknownUser_ig9cdf
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.250 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"accepting new members, healthy"


Parse progress: |█████████████████████████████████████████████████████████| 100%
AutoML progress: |████████████████
09:00:14.342: Skipping training of model GBM_5_AutoML_20201215_085939 due to exception: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for GBM model: GBM_5_AutoML_20201215_085939.  Details: ERRR on field: _min_rows: The dataset size is too small to split for min_rows=100.0: must have at least 200.0 (weighted) rows, but have only 141.0.


████████████████████████████████████████| 100%


model_id,mean_per_class_error,logloss,rmse,mse
GBM_grid__1_AutoML_20201215_085939_model_8,0.00584795,0.0751982,0.121705,0.0148121
XRT_1_AutoML_20201215_085939,0.00584795,0.12924,0.160409,0.0257312
GBM_grid__1_AutoML_20201215_085939_model_2,0.00584795,0.065775,0.106143,0.0112664
XGBoost_grid__1_AutoML_20201215_085939_model_13,0.0116959,0.146411,0.174324,0.0303888
DRF_1_AutoML_20201215_085939,0.0116959,0.117959,0.160374,0.0257198
GBM_2_AutoML_20201215_085939,0.0116959,0.218535,0.2284,0.0521667
GBM_4_AutoML_20201215_085939,0.0116959,0.161013,0.183,0.0334888
GBM_grid__1_AutoML_20201215_085939_model_6,0.0116959,0.0864015,0.127151,0.0161674
GBM_3_AutoML_20201215_085939,0.0116959,0.144264,0.174912,0.0305942
GBM_grid__1_AutoML_20201215_085939_model_3,0.0116959,0.104848,0.1453,0.0211121



Parse progress: |█████████████████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%


In [6]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, prediction['predict'])

1.0

In [7]:
from sklearn.metrics import multilabel_confusion_matrix
multilabel_confusion_matrix(y_test, prediction['predict'],labels=[1,2,3])

array([[[24,  0],
        [ 0, 12]],

       [[22,  0],
        [ 0, 14]],

       [[26,  0],
        [ 0, 10]]])

In [8]:
from sklearn.metrics import classification_report
target_names = ['1','2','3']
print(classification_report(y_test, prediction['predict'], target_names=target_names))

              precision    recall  f1-score   support

           1       1.00      1.00      1.00        12
           2       1.00      1.00      1.00        14
           3       1.00      1.00      1.00        10

    accuracy                           1.00        36
   macro avg       1.00      1.00      1.00        36
weighted avg       1.00      1.00      1.00        36



In [ ]:
import seaborn as sns
g = sns.pairplot(data, hue="Type", palette="husl")